# <center>Project 2</center>

### Task

You are in charge of leading the vaccination campaign against nonlethal disease. You have options to vactinate or provide medical treatment to infected ones. However, everything has its costs:
* Vaccination of a node costs $500 \$$ and make it immune to the disease all life-long. Unfortunately, you can help this way only to no more than $10\%$ of your population
* Medical Treatment costs $120\$$ per day of illness period, which in turn may take from $3$ to $7$ days

Your task is to implement the simulation model, propose some vaccination strategies and compare them.

### Models

Consider one of the following models:
1. SIS- and SIR- based (or another with more than 3 letters) epidemic model
2. Linear Threshold Model (threshold 15%)

Number of time steps - 30. 

**Baseline** Random vaccination or vaccination of top degree centrality vertices.

Final profit is computed as $500 \$$ * $10\%$ of population minus money spend on vaccination and curing disease.

### Data

Use PLOS [dataset](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1001109#s2) of sexual escort market. It contains buyers as well as sellers of escort services. Contains about 15.000 nodes. As it is the real-world dataset, is it interesting to test a epidemic model on it. Use SIRS model- modification of SIR model, as 'recovering' nodes return to susceptible population after some time. Also, modify model to count recovery in days, not probabilistically.

In [ ]:
from urllib.request import urlopen
import csv
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
from operator import itemgetter
from scipy.sparse import csr_matrix
import scipy.sparse.linalg as lg
from numpy.linalg import lstsq
import numpy as np
%matplotlib inline


In [2]:
dat = open('Dataset_S1.csv','rt')
reader = csv.reader(dat,delimiter=';')
Dset = list(reader)

In [3]:
g = nx.Graph(directed=False)
edgelist = []
for i in range(len(Dset)):
    if i>=24:
        edgelist.append((Dset[i][0],Dset[i][1]))
g.add_edges_from(edgelist)
g = list(nx.connected_component_subgraphs(g))[0]

Below example of use SIRS model - modification of SIR model, as our 'recovering' nodes return to susceptible population after some time.
We also modify model to count recovery in days, not probabilistically.

\begin{equation}
   \begin{cases}
   \cfrac{ds_i(t)}{dt} = -\beta s_i(t)\sum\limits_j A_{ij} x_j(t) + \gamma r_i(t) \\ 
   \cfrac{dx_i(t)}{dt} = \beta s_i(t)\sum\limits_j A_{ij} x_j(t) - \gamma x_i(t)\\
   \cfrac{dr_i(t)}{dt} = \gamma x_i(t) - \gamma r_i(t)
  \end{cases}
  \\
  x_i(t) + s_i(t) + r_i(t) = 1
\end{equation}

In [4]:
# may be used to make graph more dinamic if needed, as average degree is not very high
def random_edge(g,t):
    while i<=t:
        possible_nodes = set(g.nodes())
        first_node = choice(g.nodes())
        neighbours = g.neighbors(first_node) + [first_node]
        possible_nodes.difference_update(neighbours)
        second_node = choice(list(possible_nodes))
        g.add_edge(first_node,second_node)

**Default parameters for Epidemic modelling**

In [90]:
#creating sorted adjacency matrix to iterate
#it is sorted by node degree-will be used later
nodelist = g.nodes()
deg = nx.degree(g)
sorted_deg = sorted(deg.items(),key=itemgetter(1),reverse=True)
sortlist =[a[0]for a in sorted_deg]
A = np.array(nx.adjacency_matrix(g,nodelist = sortlist).todense())


In [25]:
#defining randomiser for infection spread
#also choosing number of initially infected individuals
idx = np.random.choice(len(nodelist),20)
I = np.zeros(len(nodelist))
R = np.zeros(len(nodelist))
S = np.ones(len(nodelist))
I[idx]=1
S[idx]=0
def lottery(y,alph):
    V = np.zeros(len(y))
    for i in range(len(y)):
        x = random.random()
        if x < (1-(1-alph)**y[i]):
            V[i] = 1
        else:
            V[i] = 0
    return V
del nodelist, deg, sorted_deg,sortlist

In [29]:
#function for time-based recovery:
#the infection now takes from 3 to 7 days to cure
def timestamp(y,Y):
    V = np.zeros(len(y))
    for i in range(len(y)):
        V[i] = random.randrange(3,7,1)*y[i]
    D=np.zeros(len(y))
    C=np.zeros(len(y))
    for j in range(len(y)):
        if Y[j]>1:
            D[j] = Y[j]-1
        elif Y[j]==1:
            C[j] = 1
    D = [x + y for x, y in zip(D, V)]
    return([D,C])
   

In [10]:
#our iterative epidemic function, based on model
def epidemic(bet,gam,delt,R,I,S,A,T):
    t=1
    rnd = list()
    R = np.zeros(len(I))
    di = I
    while t <=T:
        D = csr_matrix(A)*csr_matrix(np.diag(S))
        ss_matr = (csr_matrix(I)*D).todense().tolist()[0]
        ds = lottery(R,delt)
        dr = lottery(I,gam)
        di = lottery(ss_matr,bet)
        dI = np.subtract(di,dr)
        dS = np.subtract(ds,di)
        dR = np.subtract(dr,ds)
        I = I+dI
        S = S+dS
        R = R+dR
        rn = [sum(I),sum(S),sum(R)]
        rnd.append([round(x/len(I),2) for x in rn])
        t+=1
    return(rnd)
    


**Three competition scenarios for model parameters**

In [20]:
T1 = epidemic(0.3,0.1,0.1,R,I,S,A,30)

In [28]:
T2 = epidemic(0.2,0.5,0.2,R,I,S,A,30)

In [11]:
T3 = epidemic(0.7,0.05,0.5,R,I,S,A,30)

Here, we compute and review different ways the infection can spread, given different parameters  

In [ ]:
def epidemic_healed(bet,gam,R,I,S,A,T,h,p):
   ##put your code here
return(rnd)    

In [91]:
T4_h =(epidemic_healed(0.2,0.1,R,I,S,A,25,1,3))

In [100]:
T4_h3 = (epidemic_healed(0.2,0.1,R,I,S,A,25,1,5))

In [93]:
T4_h2 =(epidemic_healed(0.2,0.1,R,I,S,A,25,1,10))